# TF-IDF Analyse und Information Retrieval

In diesem Notebook implementieren wir ein TF-IDF basiertes IR-System und analysieren dessen Funktionsweise.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_style("whitegrid")

## 1. Daten laden

In [ ]:
data_dir = Path('../data')
csv_path = data_dir / 'tagesschau_2023_prepared.csv'

df = pd.read_csv(csv_path)
print(f"Geladene Artikel: {len(df)}")

# Identifiziere Text-Spalte
text_column = None
for col in ['text', 'content', 'article', 'body']:
    if col in df.columns:
        text_column = col
        break

if text_column is None:
    text_column = df.columns[0]

documents = df[text_column].dropna().tolist()
print(f"Verwendete Text-Spalte: {text_column}")
print(f"Anzahl Dokumente: {len(documents)}")

## 2. TF-IDF Vectorizer initialisieren

### Verständnisfragen:
1. Was bedeutet `fit_transform`?
2. Wie wird TF-IDF in scikit-learn berechnet?
3. Welche Normalisierungsoptionen gibt es?

In [ ]:
# Initialisiere TF-IDF Vectorizer
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,
    lowercase=True,
    stop_words=None
)

print("TF-IDF Vectorizer Parameter:")
print(f"  max_features: {vectorizer.max_features}")
print(f"  ngram_range: {vectorizer.ngram_range}")
print(f"  min_df: {vectorizer.min_df}")
print(f"  max_df: {vectorizer.max_df}")
print(f"  norm: {vectorizer.norm}")
print(f"  use_idf: {vectorizer.use_idf}")

## 3. TF-IDF Matrix berechnen

### Was passiert bei `fit_transform`?
- **fit**: Lernt das Vokabular aus allen Dokumenten und berechnet IDF-Werte
- **transform**: Wendet die gelernten Transformationen an und erstellt TF-IDF Vektoren

In [ ]:
# Berechne TF-IDF Matrix
print("Berechne TF-IDF Matrix...")
tfidf_matrix = vectorizer.fit_transform(documents)

print(f"\nTF-IDF Matrix Shape: {tfidf_matrix.shape}")
print(f"  Anzahl Dokumente: {tfidf_matrix.shape[0]}")
print(f"  Anzahl Features: {tfidf_matrix.shape[1]}")
print(f"  Matrix-Dichte: {tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]):.4f}")

# Zeige Feature-Namen
feature_names = vectorizer.get_feature_names_out()
print(f"\nErste 20 Features: {feature_names[:20].tolist()}")

## 4. IDF-Werte analysieren

In [ ]:
# Zeige IDF-Werte
idf_values = vectorizer.idf_

print(f"IDF-Werte:")
print(f"  Min: {idf_values.min():.4f}")
print(f"  Max: {idf_values.max():.4f}")
print(f"  Mean: {idf_values.mean():.4f}")
print(f"  Median: {np.median(idf_values):.4f}")

# Top Features nach IDF (seltene Wörter)
top_idf_indices = np.argsort(idf_values)[-20:][::-1]
print(f"\nTop 20 Features nach IDF (seltene Wörter):")
for idx in top_idf_indices:
    print(f"  {feature_names[idx]}: {idf_values[idx]:.4f}")

# Bottom Features nach IDF (häufige Wörter)
bottom_idf_indices = np.argsort(idf_values)[:20]
print(f"\nBottom 20 Features nach IDF (häufige Wörter):")
for idx in bottom_idf_indices:
    print(f"  {feature_names[idx]}: {idf_values[idx]:.4f}")

## 5. Visualisierung der IDF-Verteilung

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(idf_values, bins=50, edgecolor='black')
plt.xlabel('IDF-Wert')
plt.ylabel('Anzahl Features')
plt.title('Verteilung der IDF-Werte')
plt.axvline(idf_values.mean(), color='r', linestyle='--', label=f'Mean: {idf_values.mean():.4f}')
plt.legend()

plt.subplot(1, 2, 2)
plt.boxplot(idf_values)
plt.ylabel('IDF-Wert')
plt.title('Boxplot der IDF-Werte')

plt.tight_layout()
plt.show()

## 6. Suchfunktion implementieren

In [ ]:
def search(query, top_k=10):
    """
    Sucht nach ähnlichen Dokumenten
    """
    # Transformiere Query zu TF-IDF Vektor
    query_vector = vectorizer.transform([query])
    
    # Berechne Kosinus-Ähnlichkeit
    similarities = cosine_similarity(query_vector, tfidf_matrix)[0]
    
    # Sortiere nach Ähnlichkeit
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = [(idx, similarities[idx]) for idx in top_indices if similarities[idx] > 0]
    
    return results

## 7. Test-Suchen durchführen

In [ ]:
test_queries = [
    "Klimawandel",
    "Ukraine Krieg",
    "Bundesregierung",
    "Wirtschaft"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Suche: '{query}'")
    print(f"{'='*60}")
    
    results = search(query, top_k=5)
    
    if not results:
        print("Keine Ergebnisse gefunden.")
        continue
    
    for i, (idx, score) in enumerate(results, 1):
        print(f"\n--- Ergebnis {i} (Score: {score:.4f}) ---")
        doc = documents[idx]
        preview = doc[:300] + "..." if len(doc) > 300 else doc
        print(f"Index: {idx}")
        print(f"Text: {preview}")

## 8. Analyse und Verbesserungsvorschläge

### Diskussionspunkte:
1. Wie finden Sie das IR-System?
2. Was wären nächste Schritte zur Verbesserung?
3. Welche Vorverarbeitungsschritte könnten helfen?
4. Welche Informationen sind noch im Datensatz enthalten?

In [ ]:
# Analysiere Metadaten im Datensatz
print("Verfügbare Spalten im Datensatz:")
for col in df.columns:
    print(f"  - {col}: {df[col].dtype}")
    if df[col].dtype == 'object':
        unique_count = df[col].nunique()
        print(f"    Unique Werte: {unique_count}")
        if unique_count < 20:
            print(f"    Werte: {df[col].unique()[:10].tolist()}")